# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
                     
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
                     
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
                     
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [8]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
             
"tableName": "clients",
                     
  "fields": [
    {
      "name": "client_id",
      "type": "int"
    },
    {
      "name": "first_name",
      "type": "varchar"
    },
    {
      "name": "last_name",
      "type": "varchar"
    }
  ]

"tableName": "memberships",
                     
  "fields": [
    {
      "name": "client_id",
      "type": "int"
    },
    {
      "name": "days_left",
      "type": "int"
    },
    {
      "name": "first_name",
      "type": "varchar"
    }
  ]

"tableName": "locations",
                     
  "fields": [
    {
      "name": "client_id",
      "type": "int"
    },
    {
      "name": "country",
      "type": "varchar"
    },
    {
      "name": "city",
      "type": "varchar"
    },
    {
      "name": "street",
      "type": "varchar"
    }
  ]
"""} ]



In [10]:
# FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
Give me a qury such that I can find the days left of clients membership
"""
})

In [11]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key = OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = newcontext,
            temperature = 0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [12]:
# new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

SELECT c.first_name, c.last_name, m.days_left
FROM clients c
JOIN memberships m ON c.first_name = m.first_name;


In [ ]:
# old
old_context_user = old_context.copy()
print(return_CCRMSQL("Give me a query such that I can find the institutions and their educational level.", old_context_user))

This is your SQL:
```sql
SELECT Institution, educational_level
FROM studies;
```

This SQL query selects the columns "Institution" and "educational_level" from the "studies" table, allowing you to see the institutions and their corresponding educational levels.


In [20]:
# new
print(return_CCRMSQL("Give me a query such that I can find the days left of clients membership", context_user))

To find the days left of clients' membership, you would typically need to join the "clients" table with the "memberships" table on a common field, such as the client's first name. Here is a sample query that achieves this:

```sql
SELECT c.first_name, c.last_name, m.days_left
FROM clients c
JOIN memberships m ON c.first_name = m.first_name;
```

This query selects the first name and last name of clients along with the days left in their membership. It joins the "clients" table with the "memberships" table based on the common field "first_name".


In [ ]:
# old
print(return_CCRMSQL("Give me a query such that I can find the median of the educational level of the users.", old_context_user))

This is your SQL:
```sql
SELECT educational_level
FROM (
    SELECT educational_level,
           ROW_NUMBER() OVER (ORDER BY educational_level) AS RowAsc,
           ROW_NUMBER() OVER (ORDER BY educational_level DESC) AS RowDesc
    FROM studies
) AS subquery
WHERE RowAsc = RowDesc
   OR RowAsc + 1 = RowDesc
   OR RowAsc = RowDesc + 1;
```

This query calculates the median of the educational level of the users by ordering the educational levels in ascending and descending order, then selecting the row that falls in the middle or the average of the two middle rows.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [23]:
print(return_CCRMSQL("""Give me a query such that I can find the days left of clients membership by city""", context_user))

To find the days left of clients' membership by city, you can use the following SQL query:

```sql
SELECT c.first_name, c.last_name, m.days_left, l.city
FROM clients c
JOIN memberships m ON c.first_name = m.first_name
JOIN locations l ON c.city = l.city;
```

This query will retrieve the first name, last name, days left of membership, and city of the clients. It assumes that the `first_name` in the `memberships` table corresponds to the `first_name` in the `clients` table, and that the `city` in the `clients` table corresponds to the `city` in the `locations` table.


In [25]:
print(return_CCRMSQL("""Give me a query such that I can find the clients with membership in more than one city""", context_user))

To find the clients with memberships in more than one city, you can use the following SQL query:

```sql
SELECT c.first_name, c.last_name
FROM clients c
JOIN memberships m ON c.first_name = m.first_name
GROUP BY c.first_name, c.last_name
HAVING COUNT(DISTINCT m.city) > 1;
```

This query will retrieve the first name and last name of clients who have memberships in more than one city.


In [26]:
print(return_CCRMSQL("""Give me a query such that I can find the clients with no membership""", context_user))

```sql
SELECT c.first_name, c.last_name
FROM clients c
LEFT JOIN memberships m ON c.first_name = m.first_name
WHERE m.first_name IS NULL;
```


In [ ]:
# Report Writing
print("""It creates the queries with accuracy to whatI ask it to.""")